In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
SMOOTHING_RADIUS=50

def movingAverage(curve, radius): 
    window_size = 2 * radius + 1 
    f = np.ones(window_size)/window_size 
    curve_pad = np.lib.pad(curve, (radius, radius), 'edge') 
    curve_smoothed = np.convolve(curve_pad, f, mode='same') 
    curve_smoothed = curve_smoothed[radius:-radius]
    return curve_smoothed 

def smooth(trajectory): 
    smoothed_trajectory = np.copy(trajectory) 
    for i in range(3):
        smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)
    return smoothed_trajectory

def fixBorder(frame):
    s = frame.shape
    T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
    frame = cv2.warpAffine(frame, T, (s[1], s[0]))
    return frame

print("-------------------------------------------------------------------------------")
cp = cv2.VideoCapture("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Original_video.avi")
print("capture= ",cp)
      
n_frames = int(cp.get(cv2.CAP_PROP_FRAME_COUNT))
print("n_frames= ",n_frames)

width = int(cp.get(cv2.CAP_PROP_FRAME_WIDTH)) 
height = int(cp.get(cv2.CAP_PROP_FRAME_HEIGHT))

print("width= ", width)
print("height= ", height)

#capture.set(cv2.CAP_PROP_FPS, fps)
fps = cp.get(cv2.CAP_PROP_FPS)
print("fps= ",fps)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
print("Fourcc= ",fourcc)
#out = cv2.VideoWriter('Tracking_Railway_lines.avi',fourcc, fps, (1280, 360),True)
transforms = np.zeros((n_frames-1, 3), np.float32)


for i in range(n_frames-1):
    _, prev = cp.read()
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=10, blockSize=3)
    #cv2.imshow("feature_points",prev_pts)
    #out.write(prev_pts)
    #print("prev_points= ",prev_pts)
    succ, curr = cp.read()
    if not succ:
        break

    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)
    #print("curr_pts,status,err =",curr_pts,status,err)
    idx = np.where(status==1)[0]
    #print("index = ",idx)
    prev_pts = prev_pts[idx]
    #print("prev_points= ",prev_pts)
    curr_pts = curr_pts[idx]
    #print("curr_points= ",curr_pts)
    assert prev_pts.shape == curr_pts.shape
    #print("prev_points_shape= ",prev_pts.shape)
    #print("curr_points_shape= ",curr_pts.shape)
    m = cv2.estimateRigidTransform(prev_pts, curr_pts, fullAffine=False)
    #print("M= ",m)
    dx = m[0,2]
    #print("dx= ",dx)
    dy = m[1,2]
    #print("dy= ",dy)
    da = np.arctan2(m[1,0], m[0,0])
    
    #print("da= ",da)
    transforms[i] = [dx,dy,da]
    #plt.plot(transforms[i])
    #plt.show()
    #print("Transforms[i]= ",transforms[i])
    prev_gray = curr_gray
    #print("prev_gray= ",prev_gray)
    print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

#print("-------------------------------------------------------------------------------")
trajectory = np.cumsum(transforms, axis=0) 
#print("Trajectory= ",trajectory)
smoothed_trajectory = smooth(trajectory)
#plt.plot(trajectory,smoothed_trajectory)
#plt.show()
#print("smoothed_trajectory= ",smoothed_trajectory)
difference = smoothed_trajectory - trajectory
#print("difference= ", difference)      
transforms_smooth = transforms + difference
#print("Transforms_smooth= ",transforms_smooth)
#print("-------------------------------------------------------------------------------") 
cp.set(cv2.CAP_PROP_POS_FRAMES, 0) 

for i in range(n_frames-1):

    success, frame = cp.read() 
    if not success:

        break


    dx = transforms_smooth[i,0]
    dy = transforms_smooth[i,1]
    da = transforms_smooth[i,2]
    #print("dx= ",dx)
    #print("dy= ",dy)
    #print("da= ",da)
    m = np.zeros((2,3), np.float32)
    #print("M= ",m)
    m[0,0] = np.cos(da)
    #print("m[0,0] = ",m[0,0])
    m[0,1] = -np.sin(da)
    #print("m[0,1] = ",m[0,1])
    m[1,0] = np.sin(da)
    #print("m[1,0] = ",m[1,0])
    m[1,1] = np.cos(da)
    #print("m[1,1] = ",m[1,1])
    m[0,2] = dx
    #print("m[0,2] = ",m[0,2])
    m[1,2] = dy
    #print("m[1,2] = ",m[1,2])


    frame_stabilized = cv2.warpAffine(frame, m, (width,height))
    #cv2.imshow("before",frame_stabilized)
    #print("frame_stabilized= ",frame_stabilized)
    #print("-------------------------------------------------------------------------------")
    frame_stabilized = fixBorder(frame_stabilized) 
    #print("frame_stabilized= ",frame_stabilized)
    #cv2.imshow("frame",frame_stabilized)
    hsv=cv2.cvtColor(frame_stabilized,cv2.COLOR_BGR2HSV)
    low_pink=np.array([50, 104, 144])
    up_pink=np.array([180,  255, 255 ])
    
    Mask=cv2.inRange(frame_stabilized,low_pink,up_pink)
    edges=cv2.Canny(Mask,75,150)
    lines=cv2.HoughLinesP(edges,1,np.pi/180,20)
    
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line[0]
            cv2.line(frame_stabilized,(x1,y1),(x2,y2),(0,255,0),2)
    #print(lines)

    frame_out = cv2.hconcat([frame, frame_stabilized])
    #print("frame_out= ",frame_out)

    if(frame_out.shape[1] > 1920):
        #print("frame_out_shape[1]= ",frame_out.shape[1])
        frame_out = cv2.resize(frame_out, (frame_out.shape[1]//2, frame_out.shape[0]//2));
        #print("frame_out= ",frame_out)
        frame_out.size
        frame_out.shape
        print("frame_out_size= ",frame_out.size)
        print("frame_out_shape= ",frame_out.shape)

        cv2.imshow("Before and After", frame_out)
        #out.write(frame_out)
        cv2.waitKey(10)


cp.release()
#out.release()
cv2.destroyAllWindows()


-------------------------------------------------------------------------------
capture=  <VideoCapture 0000020B43D87EF0>
n_frames=  1937
width=  1280
height=  720
fps=  30.0
Fourcc=  1145656920
Frame: 0/1937 -  Tracked points : 198
Frame: 1/1937 -  Tracked points : 199
Frame: 2/1937 -  Tracked points : 192
Frame: 3/1937 -  Tracked points : 199
Frame: 4/1937 -  Tracked points : 200
Frame: 5/1937 -  Tracked points : 197
Frame: 6/1937 -  Tracked points : 198
Frame: 7/1937 -  Tracked points : 200
Frame: 8/1937 -  Tracked points : 199
Frame: 9/1937 -  Tracked points : 194
Frame: 10/1937 -  Tracked points : 198
Frame: 11/1937 -  Tracked points : 194
Frame: 12/1937 -  Tracked points : 197
Frame: 13/1937 -  Tracked points : 195
Frame: 14/1937 -  Tracked points : 194
Frame: 15/1937 -  Tracked points : 195
Frame: 16/1937 -  Tracked points : 199
Frame: 17/1937 -  Tracked points : 200
Frame: 18/1937 -  Tracked points : 198
Frame: 19/1937 -  Tracked points : 188
Frame: 20/1937 -  Tracked points : 

Frame: 203/1937 -  Tracked points : 199
Frame: 204/1937 -  Tracked points : 198
Frame: 205/1937 -  Tracked points : 200
Frame: 206/1937 -  Tracked points : 200
Frame: 207/1937 -  Tracked points : 200
Frame: 208/1937 -  Tracked points : 199
Frame: 209/1937 -  Tracked points : 200
Frame: 210/1937 -  Tracked points : 199
Frame: 211/1937 -  Tracked points : 200
Frame: 212/1937 -  Tracked points : 200
Frame: 213/1937 -  Tracked points : 200
Frame: 214/1937 -  Tracked points : 200
Frame: 215/1937 -  Tracked points : 200
Frame: 216/1937 -  Tracked points : 199
Frame: 217/1937 -  Tracked points : 200
Frame: 218/1937 -  Tracked points : 200
Frame: 219/1937 -  Tracked points : 199
Frame: 220/1937 -  Tracked points : 200
Frame: 221/1937 -  Tracked points : 196
Frame: 222/1937 -  Tracked points : 198
Frame: 223/1937 -  Tracked points : 200
Frame: 224/1937 -  Tracked points : 200
Frame: 225/1937 -  Tracked points : 200
Frame: 226/1937 -  Tracked points : 200
Frame: 227/1937 -  Tracked points : 200


Frame: 408/1937 -  Tracked points : 199
Frame: 409/1937 -  Tracked points : 200
Frame: 410/1937 -  Tracked points : 197
Frame: 411/1937 -  Tracked points : 196
Frame: 412/1937 -  Tracked points : 198
Frame: 413/1937 -  Tracked points : 199
Frame: 414/1937 -  Tracked points : 199
Frame: 415/1937 -  Tracked points : 200
Frame: 416/1937 -  Tracked points : 200
Frame: 417/1937 -  Tracked points : 199
Frame: 418/1937 -  Tracked points : 198
Frame: 419/1937 -  Tracked points : 200
Frame: 420/1937 -  Tracked points : 199
Frame: 421/1937 -  Tracked points : 200
Frame: 422/1937 -  Tracked points : 200
Frame: 423/1937 -  Tracked points : 200
Frame: 424/1937 -  Tracked points : 200
Frame: 425/1937 -  Tracked points : 199
Frame: 426/1937 -  Tracked points : 198
Frame: 427/1937 -  Tracked points : 192
Frame: 428/1937 -  Tracked points : 199
Frame: 429/1937 -  Tracked points : 196
Frame: 430/1937 -  Tracked points : 200
Frame: 431/1937 -  Tracked points : 197
Frame: 432/1937 -  Tracked points : 198


Frame: 613/1937 -  Tracked points : 198
Frame: 614/1937 -  Tracked points : 200
Frame: 615/1937 -  Tracked points : 198
Frame: 616/1937 -  Tracked points : 198
Frame: 617/1937 -  Tracked points : 197
Frame: 618/1937 -  Tracked points : 198
Frame: 619/1937 -  Tracked points : 196
Frame: 620/1937 -  Tracked points : 199
Frame: 621/1937 -  Tracked points : 197
Frame: 622/1937 -  Tracked points : 196
Frame: 623/1937 -  Tracked points : 195
Frame: 624/1937 -  Tracked points : 195
Frame: 625/1937 -  Tracked points : 194
Frame: 626/1937 -  Tracked points : 200
Frame: 627/1937 -  Tracked points : 195
Frame: 628/1937 -  Tracked points : 193
Frame: 629/1937 -  Tracked points : 196
Frame: 630/1937 -  Tracked points : 197
Frame: 631/1937 -  Tracked points : 198
Frame: 632/1937 -  Tracked points : 194
Frame: 633/1937 -  Tracked points : 190
Frame: 634/1937 -  Tracked points : 188
Frame: 635/1937 -  Tracked points : 194
Frame: 636/1937 -  Tracked points : 200
Frame: 637/1937 -  Tracked points : 194


Frame: 820/1937 -  Tracked points : 199
Frame: 821/1937 -  Tracked points : 200
Frame: 822/1937 -  Tracked points : 196
Frame: 823/1937 -  Tracked points : 200
Frame: 824/1937 -  Tracked points : 197
Frame: 825/1937 -  Tracked points : 197
Frame: 826/1937 -  Tracked points : 198
Frame: 827/1937 -  Tracked points : 200
Frame: 828/1937 -  Tracked points : 190
Frame: 829/1937 -  Tracked points : 194
Frame: 830/1937 -  Tracked points : 186
Frame: 831/1937 -  Tracked points : 194
Frame: 832/1937 -  Tracked points : 200
Frame: 833/1937 -  Tracked points : 200
Frame: 834/1937 -  Tracked points : 200
Frame: 835/1937 -  Tracked points : 200
Frame: 836/1937 -  Tracked points : 200
Frame: 837/1937 -  Tracked points : 196
Frame: 838/1937 -  Tracked points : 200
Frame: 839/1937 -  Tracked points : 200
Frame: 840/1937 -  Tracked points : 199
Frame: 841/1937 -  Tracked points : 199
Frame: 842/1937 -  Tracked points : 199
Frame: 843/1937 -  Tracked points : 198
Frame: 844/1937 -  Tracked points : 199


...

frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size

frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size

frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size

frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size

frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size=  1382400
frame_out_shape=  (360, 1280, 3)
frame_out_size